In [1]:
from googlefinance import getQuotes
import sys

### email function

In [2]:
import getpass
import smtplib
from email.MIMEMultipart import MIMEMultipart
from email.MIMEBase import MIMEBase
from email.MIMEText import MIMEText
from email import Encoders
import os

# reference
# https://gist.github.com/dbieber/5146518

In [3]:
gmail_user = 'ylm.neu@gmail.com'
gmail_pwd = 'r9bbtzhy'

In [4]:
def mail(to, subject, text, attach=None):
    global gmail_user, gmail_pwd
    msg = MIMEMultipart()
    msg['From'] = gmail_user
    msg['To'] = to
    msg['Subject'] = subject
    msg.attach(MIMEText(text))
    if attach:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(open(attach, 'rb').read())
        Encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(attach))
        msg.attach(part)
    mailServer = smtplib.SMTP("smtp.gmail.com", 587)
    mailServer.ehlo()
    mailServer.starttls()
    mailServer.ehlo()
    mailServer.login(gmail_user, gmail_pwd)
    mailServer.sendmail(gmail_user, to, msg.as_string())
    mailServer.close()

In [5]:
# try:
#     mail("leimingyu830@gmail.com", "雷鸣友情提醒", "test!")
#     print 'Email sent!'
    
# except:
#     print 'Something went wrong...'

### google finance for stock price

In [6]:
target_list = ['AMD', 'QCOM', 'NVDA']

In [7]:
# basic info
buy_price_dd = {}
buy_volume_dd = {}

buy_price_dd['AMD'] = 13.105
buy_volume_dd['AMD'] = 200

buy_price_dd['QCOM'] = 61.8246
buy_volume_dd['QCOM'] = 500

buy_price_dd['NVDA'] = 100.8448
buy_volume_dd['NVDA'] = 200

In [8]:
for stock_name in target_list:
    result = getQuotes(stock_name)
    current_time = result[0]['LastTradeDateTimeLong']
    current_price = float(result[0]['LastTradePrice'])
    
    my_price = buy_price_dd[stock_name]
    my_volume = buy_volume_dd[stock_name]
    
    gainSell_price = my_price * 1.13 # increase 13%
    loseSell_price = my_price * 0.99 # drop 10%
    
    if current_price >= gainSell_price or current_price <= loseSell_price:
        send_msg = stock_name + ' : \n成本 ' + str(my_price) +  ', \n现价 ' + str(current_price) + '\n'
        send_msg += str(current_time)
        try:
            mail("leimingyu830@gmail.com", "股市提醒", send_msg)
        except:
            sys.stderr.write('Something went wrong...')